In [1]:
# %load 05_final_inputlas_opcurvekey_outputtwocolumns.py


    def _save_las(self, fileName):
        """Save a LAS file.

        Returns a dictionary with keys 'V', 'W', 'C', 'P', 'O' and 'A',
        corresponding to the sections of a LAS file.  The values associated
        with keys 'V', 'W', 'C' and 'P' will be lists of Item instances.  The
        value associated with the 'O' key is a list of strings.  The value
        associated with the 'A' key is a numpy structured array containing the
        log data.  The field names of the array are the mnemonics from the
        Curve section of the file.
        """
        from datetime import datetime

        with open(fileName, 'w+') as fout:
            fout.write(
                """#------------------------------------------------------------------------------
#
#   Created by  : 
#   Created on  : {date}
#
#------------------------------------------------------------------------------
""".format(date=datetime.now().strftime('%Y/%m/%d %H:%M:%S')))

            # write veriosn
            wrap = 'YES' if self.wrap else 'NO'
            fout.write(
                """~VERSION INFORMATION
VERS. {vers} : CWLS Log ASCII Standard - version {vers}
WRAP. {wrap:{fill}{align}3} : One line per depth step
""".format(vers=self.vers, wrap=wrap, fill=' ', align='>'))

            # write well information
            fout.write(
                """~WELL INFORMATION
{name:12}.{units:12}{data:>40} :{descr}
#------------------------------------------------------------------------------
""".format(name='#MNEMONIC', units='UNIT', data='VALUE', descr='DESCRIPTION', fill=' ', align='>'))

            # print self.well.names
            # print self.well.items
            for item_name in self.well.names:
                item = self.well.items[item_name]
                if item is not None:
                    fout.write('{name:12}.{units:12}{data:>40} :{descr}\n'.format(
                        name=item.name, units=item.units, data=item.data, descr=item.descr, fill=' ', align='>'))

            # write curve information
            fout.write(
                """~CURVE INFORMATION
{name:11}.{units:12}{data:14} : {descr}
#------------------------------------------------------------------------------
""".format(name='#MNEMONIC', units='UNIT', data='API CODE', descr='CURVE DESCRIPTION', fill=' ', align='>'))

            # print self.curves.names
            # print self.curves.items
            for item_name in self.curves.names:
                item = self.curves.items[item_name]
                # print "api[",item.data,']',"descr[",item.descr,']'
                if item is not None:
                    fout.write('{name:11}.{units:12}{api:14} : {descr}\n'.format(
                        name=item.name, units=item.units, api=item.data, descr=item.descr))

            # write parameter information
            if len(self.parameters.names) > 0:
                fout.write(
                    """~PARAMETER INFORMATION
{name:12}.{units:12}{data:>36} :{descr}
#-------------- ---------------- ------------------------
""".format(name='#MNEMONIC', units='UNIT', data='VALUE', descr='DESCRIPTION', fill=' ', align='>'))

            for item_name in self.parameters.names:
                item = self.parameters.items[item_name]
                if item is not None:
                    fout.write('{name:12}.{units:12}{data:>36} :{descr}\n'.format(
                        name=item.name, units=item.units, data=item.data, descr=item.descr, fill=' ', align='>'))

            # write other information
            if len(self.other) > 0:
                fout.write("~OTHER INFORMATION\n{other}\n".format(
                    other=self.other))

            # write data information
            fout.write('~A')
            #size = 10
            # for item_name in self.curves.names:
            #    fout.write('{name:>{size}}'.format(name=item_name, size=size))
            #    size = 13
            fout.write('\n')

            new_data = np.copy(self.data2d)
            #new_data[np.isnan(new_data)] = self.null

            if np.isnan(self.null_subs):
                new_data[np.isnan(new_data)] = self.null
            else:
                new_data[new_data == self.null_subs] = self.null

            if self.wrap:
                self._write_wrapped_data(fout, new_data)
            else:
                np.savetxt(fout, new_data, fmt='%12.4f')


class Well():

    def __init__(self):
        self.m_source = None

        self.m_api = None
        self.m_wellid = None  # key
        self.m_name = None
        self.m_lasfile = None

        self.m_emw = {'FIT': [], 'FLOW': [], 'LOST': []}
        self.m_mw = None

        self.m_las = None

    def display(self):
        print '>>>>> wellid:', self.m_wellid

        for item in self.m_emw['FIT']:
            print "FIT", item

        for item in self.m_emw['FLOW']:
            print "FLOW", item

        for item in self.m_emw['LOST']:
            print "LOST", item

        if self.m_las is not None:
            print self.m_las.well.items['API'].data, self.m_las.well.items['WELL'].data
            print self.m_las.curves.names

    def name(self):
        name = ''
        if self.m_name is not None:
            name += self.m_name

        if name == '':
            name = self.m_wellid
        else:
            name += ' (wellid:' + str(self.m_wellid) + ')'

        return name


def createLAS(fullname):
    las = LASReader(fullname, null_subs=np.nan)
    '''
    print "wrap? ", las.wrap
    print "vers? ", las.vers
    print "null =", las.null
    print "start =", las.start
    print "stop  =", las.stop
    print "step  =", las.step
    print "Version ---"
    las.version.display()
    '''
    # print "Well ---"
    # las.well.display()
    # print "Curves ---"
    # las.curves.display()
    # print "Parameters ---"
    # las.parameters.display()
    # print "Other ---"
    # print las.other
    # print "Data ---"
    # print las.data2d

    return las


def getNamedCurve(las, curve):
    if curve in las.curves.names:
        #fignum += 1
        # plt.figure(fignum)
        #plt.title(name + ' - ' + curve)
        #ax = plt.gca()
        # ax.invert_yaxis()
        #ax.set_xlim(-3, 5)
        #ax.set_ylim(30000, 0)
        if 'DEPT' in las.curves.names:
            return las.data[curve], las.data['DEPT']
        else:
            return las.data[curve], las.data['DEPTH']

    return None, None



import sys   
if __name__ == "__main__":



    #input format: command, input las folder, optional curve keywords
    #curve_keys = ['DT','GR']
    curve_keys = []
    
    if len(sys.argv)<3:
        print 'usage: \n %s input_las_folder output_las_folder optional_curve_keyword1 optional_curve_keyword2 ... \n ' %(sys.argv[0])
        
        
    else:
        input_las_dir=sys.argv[1]
        output_las_dir = sys.argv[2]
        print input_las_dir
        print output_las_dir
        
        for i in np.arange(3,len(sys.argv)):
            curve_keys.append(sys.argv[i])
            
           
        
    
    #curve_keys.append('DT')
    well_map = {}
    well_path = input_las_dir
    
    '''read all las file in give folder 
    '''
    las_files = []
    for f in os.listdir(well_path):
        if f.lower().endswith(".las"):
            las_files.append(os.path.join(well_path, f))
    print las_files     
    
    '''Make a output folder for new two column las file
    '''
    
    #file_path = os.path.join(well_path,"new_two_column_las_withkey/")
    file_path = output_las_dir
    directory = os.path.dirname(file_path)

    try:
        os.stat(directory)
    except:
        os.mkdir(directory) 
     
        
        
           
  
    for fn in las_files:       

        #read las file content
        try:
            
            las = createLAS(fn)
        except ValueError as e:
            print "ERROR in loading", fn
            print e.args
            continue     
        # read a copy of las file content
        las_cp = createLAS(fn)

        if 'UWI' in las.well.items.keys():
            wellid = las.well.items['UWI'].data
        elif 'API' in las.well.items.keys():
            wellid = las.well.items['API'].data
 
        else:
            wellid = las.well.items['WELL'].data
        
        wellname = las.well.items['WELL'].data
        
        #las.well.display()
        #print las.well.names
        for curvename in las.curves.items.keys():
            #if curvename == 'DEPTH' or curvename == 'DEPT':
            
            if curvename.startswith('DEPT'):
                
                item = las.curves.items[curvename]
                break
        
        for curvename in las.curves.items.keys():
         
           
            if curvename.startswith('DEPT'):
                
                pass
            else:
                if curve_keys == []:
                    
                    dt_log, dt_depth = getNamedCurve(las, curvename)
                    
                    
                    idx = np.where(np.isnan(dt_log) == False)
                    
                    dt_depth = dt_depth[idx]
                    dt_log = dt_log[idx]
                    log = np.column_stack((dt_depth, dt_log))
                
                
                    
                    las_cp.curves.names = []
                    las_cp.curves.items.clear()
                    las_cp.curves.add_item(item)
                    for cname in las.curves.items.keys():
                        if cname == curvename:
                            item1 = las.curves.items[cname]
                            las_cp.curves.add_item(item1)
                            las_cp.curves.display()
                            new_lasfile = 'new_{0}'.format(wellid)
                            new_lasfile_fullpath = os.path.join(file_path,new_lasfile+'_'+curvename+'.las')
                            print "new las:", new_lasfile_fullpath
                            las_cp.data2d = log
                            print las.data2d.shape   
                            las_cp._save_las(new_lasfile_fullpath)   
                            break 
                else:
                    for key in curve_keys:
                        if curvename.startswith(key):
                        
                            dt_log, dt_depth = getNamedCurve(las, curvename)
                            
                            
                            idx = np.where(np.isnan(dt_log) == False)
                            
                            dt_depth = dt_depth[idx]
                            dt_log = dt_log[idx]
                            log = np.column_stack((dt_depth, dt_log))
                        
                        
                            
                            las_cp.curves.names = []
                            las_cp.curves.items.clear()
                            las_cp.curves.add_item(item)
                            for cname in las.curves.items.keys():
                                if cname == curvename:
                                    item1 = las.curves.items[cname]
                                    las_cp.curves.add_item(item1)
                                    las_cp.curves.display()
                                    new_lasfile = 'new_{0}'.format(wellid)
                                    new_lasfile_fullpath = os.path.join(file_path,new_lasfile+'_'+curvename+'.las')
                                    print "new las:", new_lasfile_fullpath
                                    las_cp.data2d = log
                                    print las.data2d.shape   
                                    las_cp._save_las(new_lasfile_fullpath)   
                                    break 
                                          
    print curve_keys
    print 'Finished All!'
    